In [406]:
# 내가 DT를 짠것과 RF가 앙상블 한것이 어떤 차이가 있는지
# 나만의 Decision tree를 여러개 구성해서 예측한 것과, 랜덤포레스트로 예측한 것의 결과 비교
# 각 개성을 가진 DT 100개가 투표를 해서 x번 이상 투표 된것만 골라보기


In [407]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import KFold


from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns



In [408]:
# 사용할 변수 정의
number_of_tree = 10

In [409]:
#사용할 함수 정의

# Null 처리 함수
def fillna(df):
    df['Age'].fillna(df['Age'].mean(),inplace=True)
    df['Cabin'].fillna('N',inplace=True)
    df['Embarked'].fillna('N',inplace=True)
    df['Fare'].fillna(0,inplace=True)
    return df

# 머신러닝 알고리즘에 불필요한 속성 제거
def drop_features(df):
    df.drop(['PassengerId','Name','Ticket'],axis=1,inplace=True)
    return df

# 레이블 인코딩 수행. 
def format_features(df):
    df['Cabin'] = df['Cabin'].str[:1]
    features = ['Cabin','Sex','Embarked']
    for feature in features:
        le = LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df

# 앞에서 설정한 Data Preprocessing 함수 호출
def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df

# k폴드 함수 선언
def exec_kfold(clf, folds=5):
    # 폴드 세트를 5개인 KFold객체를 생성, 폴드 수만큼 예측결과 저장을 위한  리스트 객체 생성.
    kfold = KFold(n_splits=folds)
    scores = []
    
    # KFold 교차 검증 수행. 
    for iter_count , (train_index, test_index) in enumerate(kfold.split(X_titanic_df)):
        # X_titanic_df 데이터에서 교차 검증별로 학습과 검증 데이터를 가리키는 index 생성
        X_train, X_test = X_titanic_df.values[train_index], X_titanic_df.values[test_index]
        y_train, y_test = y_titanic_df.values[train_index], y_titanic_df.values[test_index]
        
        # Classifier 학습, 예측, 정확도 계산 
        clf.fit(X_train, y_train) 
        predictions = clf.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        scores.append(accuracy)
        print("교차 검증 {0} 정확도: {1:.4f}".format(iter_count, accuracy))     
    
    # 5개 fold에서의 평균 정확도 계산. 
    mean_score = np.mean(scores)
    print("평균 정확도: {0:.4f}".format(mean_score)) 

In [410]:
#데이터 재로딩 및 전처리

train = pd.read_csv("./Data/train.csv")
test = pd.read_csv("./Data/test.csv")




In [411]:
train.Age.fillna(train.Age.mean() , inplace = True)
train.Embarked.fillna("S" , inplace =True)
test.Age.fillna(test.Age.mean() , inplace = True)
test.Fare.fillna(test.Fare.mean() , inplace = True)

In [412]:
le = LabelEncoder()

In [413]:
train.Sex = le.fit_transform(train.Sex)

In [414]:
train_oh = pd.get_dummies(train , columns= ["Embarked"])

In [415]:
features = ['Pclass', 'Sex', 'Age', 'SibSp','Parch',  'Fare', 'Embarked_C', 'Embarked_Q','Embarked_S']

In [416]:
X_train = train_oh[features]
y_train = train_oh.Survived
test_oh = pd.get_dummies(test , columns= ["Embarked"])
X_test = test_oh[features]
X_test.Sex = le.fit_transform(X_test.Sex)

/home/trisix/anaconda3/lib/python3.9/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [417]:
data_number_list = []
for i in range(0, len(X_test)):
    data_number_list.append(i)

In [418]:
data_df = pd.DataFrame(data_number_list)

In [419]:
# 나무에 각자 개성 부여해서 투표하기
for j in range(0,number_of_tree):

    dt_clf = DecisionTreeClassifier(random_state=j)
    dt_clf.fit(X_train, y_train)
    pred_dt = dt_clf.predict(X_test)
    data_df[j] = pred_dt

In [420]:
data_df['sum'] = data_df.sum(axis=1)

In [421]:
#data_df

In [422]:
#df['Status'] = np.where(df['Salary']>=400, 'Senior', 'Junior')

final_df_dt = np.where(data_df['sum']>(number_of_tree/2), 1, 0)

In [423]:
#final_df_dt

In [424]:
# 내가 dt 짜서 해본 결과
submission = pd.read_csv("./Data/gender_submission.csv" , index_col= "PassengerId")
submission.Survived = final_df_dt
submission.to_csv("./Submission/dt_ds_10_predcit_50.csv") 
# 예측 결과 : (나무 10개) 투표수 절반 이상일 때 : 
# 예측 결과 : (나무 100개)투표수 1 이상일 때 : 0.73444, 투표수 절반 이상일 때 : 0.74401, 투표수 99 이상일 때 : 0.73684
# 예측 결과 : (나무 1000개) 투표수 절반 이상일 때 : 0.74162
# 예측 결과 : (나무 10000개) 투표수 절반 이상일 때 : 0.74401

In [425]:
rf_clf = RandomForestClassifier(n_estimators = number_of_tree)
rf_clf.fit(X_train, y_train)
pred_rf = rf_clf.predict(X_test)

submission.Survived = pred_rf
submission.to_csv("./Submission/rf_ds_10_predcit.csv") 

# 예측 결과 : (나무 10개) 랜덤포레스트 n_estimators 10개로 했을 때 : 
# 예측 결과 : (나무 100개) 랜덤포레스트 n_estimators 100개로 했을 때 : 0.76315
# 예측 결과 : (나무 1000개) 랜덤포레스트 n_estimators 1000개로 했을 때 : 0.76555
# 예측 결과 : (나무 10000개) 랜덤포레스트 n_estimators 10000개로 했을 때 : 0.76555

# 최종 결과 : 
# (나무 100개) 나만의 DT : 0.74401(0.3s), 랜덤포레스트 기본값 : 0.76315(0.1s)
# (나무 1000개) 나만의 DT : 0.74162(2.8s), 랜덤포레스트 기본값 : 0.76555(1.6s)
# (나무 10000개) 나만의 DT : 0.74162(32.4s), 랜덤포레스트 기본값 : 0.76555(10.6s)